In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import git

In [2]:
# variables and files to be apply horizontal averaging to
metadata_list = [
    {'fname_in':'../work/omip1/g.e21.GOMIPECOIAF.T62_g17.CMIP6-omip1.002.pop.h.N_HEAT.0311-0372.nc',
     'fname_time':'../omip1/thetao_Oyr_CESM2_omip1_r2i1p1f1_gr_zon_avg_0311-0372.nc',
     'fname_basin':'../omip1/msftmz_Oyr_CESM2_omip1_r2i1p1f1_gn_0001-0372.nc',
     'fname_out':'../omip1/hfbasin_Oyr_CESM2_omip1_r2i1p1f1_gn_0311-0372.nc'},
    {'fname_in':'../work/omip2/g.e21.GOMIPECOIAF_JRA.TL319_g17.CMIP6-omip2.001.pop.h.N_HEAT.0306-0366.nc',
     'fname_time':'../omip2/thetao_Oyr_CESM2_omip2_r1i1p1f1_gr_zon_avg_0306-0366.nc',
     'fname_basin':'../omip2/msftmz_Oyr_CESM2_omip2_r1i1p1f1_gn_0001-0366.nc',
     'fname_out':'../omip2/hfbasin_Oyr_CESM2_omip2_r1i1p1f1_gn_0306-0366.nc'},
]

In [3]:
for metadata in metadata_list:
    ds_in = xr.open_dataset(metadata['fname_in'])
    ds_time = xr.open_dataset(metadata['fname_time'])
    ds_basin = xr.open_dataset(metadata['fname_basin'])

    ds_out = xr.Dataset()
    for varname in ['time', 'time_bnds']:
        ds_out[varname] = ds_time[varname]
    for varname in ['basin', 'lat', 'lat_bnds']:
        ds_out[varname] = ds_basin[varname]

    N_HEAT = xr.DataArray(
        ds_in.N_HEAT.values,
        dims=('time', 'transport_reg', 'transport_comp', 'lat'),
        coords={'time':ds_time.time, 'lat':ds_basin.lat})
    
    hfbasin = xr.DataArray(
        np.zeros((ds_time.dims['time'], ds_basin.dims['basin'], ds_basin.dims['lat'])),
        dims=('time', 'basin', 'lat'),
        coords={'time':ds_time.time, 'basin':ds_basin.basin, 'lat':ds_basin.lat})

    for key in ['dtype', '_FillValue', 'zlib', 'complevel']:
        if key in ds_basin.msftmz.encoding:
            hfbasin.encoding[key] = ds_basin.msftmz.encoding[key]
    if 'missing_value' in ds_basin.msftmz.encoding:
        hfbasin.attrs['missing_value'] = np.float32(ds_basin.msftmz.encoding['missing_value'])

    # basin indices
    glo_ind = {'N_HEAT':0, 'hfbasin':2}
    atl_arc_ind = {'N_HEAT':1, 'hfbasin':0}
    indo_pac_ind = {'hfbasin':1}
    
    # Eulerian advection
    for basin_ind in [glo_ind, atl_arc_ind]:
        hfbasin[:,basin_ind['hfbasin'],:] = N_HEAT[:,basin_ind['N_HEAT'],1,:].copy()

    # all parameterized advection
    for basin_ind in [glo_ind, atl_arc_ind]:
        hfbasin[:,basin_ind['hfbasin'],:] += \
            N_HEAT[:,basin_ind['N_HEAT'],3,:].copy() + N_HEAT[:,basin_ind['N_HEAT'],4,:].copy()

    # Indo-Pacific transport = Global - Atlantic-Arctic transport
    hfbasin[:,indo_pac_ind['hfbasin'],:] = \
        hfbasin[:,glo_ind['hfbasin'],:] - hfbasin[:,atl_arc_ind['hfbasin'],:]
    hfbasin[:,indo_pac_ind['hfbasin'],:] = hfbasin[:,indo_pac_ind['hfbasin'],:].where(hfbasin.lat<66.0)

    # diffusion (not computed by model for Atlantic-Arctic basin)
    for basin_ind in [glo_ind]:
        hfbasin[:,basin_ind['hfbasin'],:] += \
            N_HEAT[:,basin_ind['N_HEAT'],2,:].copy() \
            - N_HEAT[:,basin_ind['N_HEAT'],3,:].copy() \
            - N_HEAT[:,basin_ind['N_HEAT'],4,:].copy()

    hfbasin *= 1.0e15

    for key in ['long_name', 'title']:
        hfbasin.attrs[key] = 'Northward Ocean Heat Transport'
    for key in ['id', 'out_name', 'variable_id']:
        hfbasin.attrs[key] = 'hfbasin'

    hfbasin.attrs.update(
        {'units':'W', 'coordinates':'time basin lat', 'frequency':'yr',
         'realm':'ocean', 'standard_name':'northward_ocean_heat_transport'})

    hfbasin.attrs['description'] = \
    'Contains contributions from all physical processes affecting the northward heat transport, ' \
    'including resolved advection, parameterized advection, lateral diffusion, etc. ' \
    'Diagnosed here as a function of latitude and basin. Use Celsius for temperature scale.'

    hfbasin.attrs['comment'] = 'constructed by klindsay@ucar.edu from N_HEAT using hfbasin.ipynb'

    ds_out['hfbasin'] = hfbasin

    # if a variable doesn't have _FillValue in the encoding, set _FillValue to None
    for varname in ds_out.variables:
        if '_FillValue' not in ds_out[varname].encoding:
            ds_out[varname].encoding['_FillValue'] = None

    ds_out.attrs = ds_basin.attrs
    ds_out.attrs['variable_id'] = 'hfbasin'
    ds_out.attrs['contact'] = 'klindsay@ucar.edu'
    ds_out.attrs['githash'] = git.Repo(search_parent_directories=True).head.object.hexsha

    ds_out.to_netcdf(metadata['fname_out'], unlimited_dims='time')